In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-08 13:20:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.42MB/s    in 0.2s    

2022-03-08 13:20:07 (4.42 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...|2015-01-31 00:08:00|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...| 

In [5]:
df.count()

792113

In [6]:
# Drop null values
df = df.dropna()

print(df.count())

791971


In [7]:
# Transform
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [8]:
from pyspark.sql.types import TimestampType, IntegerType

df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", df["total_votes"].cast(IntegerType()))\

df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [9]:
# Create dataframe to match tables
from pyspark.sql.functions import desc
# customers
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   37141039|           142|
|   32405532|            71|
|   52277880|            44|
|   45212655|            33|
|   21957586|            33|
|   35178127|            28|
|   36700743|            27|
|   12609448|            25|
|   20845991|            25|
|   36020793|            25|
|   31511151|            22|
|   49465759|            22|
|   13278937|            22|
|   40418760|            22|
|   32479799|            21|
|   27393836|            21|
|   18469006|            21|
|   21155796|            21|
|    2408932|            20|
|   51672584|            20|
+-----------+--------------+
only showing top 20 rows



In [10]:
# products (unique values)
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0049H81OM|Fun Rugs Surf Tim...|
|B001U0UOO6|Furniture Repair Set|
|B0076I51JE|Traditional Opera...|
|B007OWPBGU|Frenchi Home Furn...|
|B00DHWGB4M|Circle Design Sid...|
|B00RI6TNJ8|Abrahami Hariz Bu...|
|B00GHZA29Q|nuLOOM Varanas Co...|
|B00GSIIGQS|Milton Greens Sta...|
|B00VZ4RY1I|Dean Shifting San...|
|B007QUM5DM| Charles Petite Sofa|
|B00MN6NTDO|Safavieh Adironda...|
|B00BK31LDQ|Glass Computer De...|
|B0091SXURW|Altra Parsons Des...|
|B00TBVK0Y0|Best Price Mattre...|
|B00A2XM5QC|Legacy Decor Shoj...|
|B002KE7HTQ|Home Styles 5001-...|
|B00V3LVD2O|Roundhill Furnitu...|
|B00PN9YSAG|Baxton Studio Hir...|
|B005VAFFN6|Duro Hanley Silve...|
|B001BX1JSC|Flash Furniture V...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# review_id_table
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3VR960AHLFKDV|   24509695|B004HB5E0E|     488241329|2015-01-31 00:08:00|
|R16LGVMFKIUT0G|   34731776|B0042TNMMS|     205864445|2015-01-31 00:08:00|
|R1AIMEEPYHMOE4|    1272331|B0030MPBZ4|     124663823|2015-01-31 00:08:00|
|R1892CCSZWZ9SR|   45284262|B005G02ESA|     382367578|2015-01-31 00:08:00|
|R285P679YWVKD1|   30003523|B005JS8AUA|     309497463|2015-01-31 00:08:00|
| RLB33HJBXHZHU|   18311821|B00AVUQQGQ|     574537906|2015-01-31 00:08:00|
|R1VGTZ94DBAD6A|   42943632|B00CFY20GQ|     407473883|2015-01-31 00:08:00|
|R168KF82ICSOHD|   43157304|B00FKC48QA|     435120460|2015-01-31 00:08:00|
|R20DIYIJ0OCMOG|   51918480|B00N9IAL9K|     356495985|2015-01-31 00:08:00|
| RD46RNVOHNZSC|   14522766|B001T4XU1C|     243050228|2015-01-31 00:08:00|
|R2JDOCETTM3AXS|   430541

In [12]:
# vine_table table
# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# Filter for Amazon's Vine program
vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1GJOV4QXAVTH3|          2|            0|          1|   Y|
|R16IQLIZFO0CAW|          4|            0|          0|   Y|
|R12EEO0JZ9K47J|          3|            0|          0|   Y|
|R3LBZ21KGMXFKV|          4|            1|          3|   Y|
|R3EVTESN1DYY36|          5|            0|          0|   Y|
|R2BQOD1R0228FN|          3|           17|         26|   Y|
|R3JX1GD08BQ475|          4|            0|          0|   Y|
|R1XBF1CCX5A99R|          5|            0|          0|   Y|
|R23CARBC5CFO0Z|          5|            1|          1|   Y|
|R38UKMX18XD6NJ|          5|            0|          0|   Y|
|R3C4PCIB2GSIVH|          4|            5|          6|   Y|
|R2VAR4Z3X91OTF|          4|            2|          2|   Y|
| RK686JE1N7LRU|          4|            2|          2|   Y|
|R2HUFRZTFLPBXF|          4|            

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>.rds.amazonaws.com:5432/bigdata_db"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)



In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)



In [ ]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)



In [ ]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
